In [1]:
# install neccessary packages
!pip install stable-baselines3[extra]
!pip install gym[all]

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Using cached lz4-4.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (42.3 MB)
  Using cached Cython-0.29.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached imageio-2.19.0-py3-none-any.whl (3.4 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Using setuptools (version 59.6.0).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.10
      creating build/lib.linux-x86_64-3.10/Box2D
      copying library/Box2D/__init__.py -> build/lib.linux-x86_64-3.10/Box2D
      copying

# 1. Import dependencies

In [4]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/home/pooya-fallah/python_envs/reinforce_practice/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load Enviroment

In [5]:
enviroment_name = "CartPole-v0"

In [6]:
env = gym.make(enviroment_name)

In [14]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:33.0
Episode:2 Score:30.0
Episode:3 Score:20.0
Episode:4 Score:10.0
Episode:5 Score:44.0


# 3. Train an RL Model

In [7]:
log_path = os.path.join("Training", "logs")

In [8]:
env = gym.make(enviroment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [9]:
model.learn(total_timesteps=20000)

Logging to Training/logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 2401 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1686        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010851232 |
|    clip_fraction        | 0.138       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.00202     |
|    learning_rate        | 0.0003      |
|    loss                 | 9.98        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0214     |
|    value_loss           | 49.5        |
-----------------------------------------
---

# 4. Save and Reload Model

In [7]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_CartPole')

In [30]:
model.save(PPO_Path)

/home/pooya-fallah/python_envs/reinforce_practice/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [8]:
# Load Model
model = PPO.load(PPO_Path, env=env)

# 5. Evaluation

In [36]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [39]:
env.close()

# 6. Test Model

In [9]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


# 7. Viewing Logs in Tensorboard

In [42]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [43]:
!tensorboard --logdir={training_log_path}

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# 8. Adding a callback to the training stage

In [28]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [29]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'logs')

In [30]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1)

In [31]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [32]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 2244 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1611         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0085326955 |
|    clip_fraction        | 0.0868       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.00157     |
|    learning_rate        | 0.0003       |
|    loss                 | 7.64         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0143      |
|    value_loss           | 52.8         |
----------------------------

/home/pooya-fallah/python_envs/reinforce_practice/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0069118235 |
|    clip_fraction        | 0.0515       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.611       |
|    explained_variance   | 0.179        |
|    learning_rate        | 0.0003       |
|    loss                 | 34.6         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0136      |
|    value_loss           | 70.8         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 190


In [25]:
model_path = os.path.join('Trainig', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

0

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [33]:
env.close()

# 9. Changing Policies

In [34]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [35]:
model = PPO('MlpPolicy', env, verbose=1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [36]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1773 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1227        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015329748 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00553    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.09        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0253     |
|    value_loss           | 17.8        |
-----------------------------------------
----------------------------------

/home/pooya-fallah/python_envs/reinforce_practice/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.013776025 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.567      |
|    explained_variance   | 0.388       |
|    learning_rate        | 0.0003      |
|    loss                 | 6.94        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0181     |
|    value_loss           | 36.5        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 986   |
|    iterations      | 5     |
|    time_elapsed    | 10    |


# 10. Using an Alternate Algorithm

In [37]:
from stable_baselines3 import DQN

In [38]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [39]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.957    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 110      |
|    time_elapsed     | 0        |
|    total_timesteps  | 90       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.907    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 238      |
|    time_elapsed     | 0        |
|    total_timesteps  | 196      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.871    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 328      |
|    time_elapsed     | 0        |
|    total_timesteps  | 272      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 2311     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2384     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 2352     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2439     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 2396     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2500     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 3731     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4639     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 3788     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4754     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 3813     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4809     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 4775     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7075     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 4799     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7148     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 4848     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7276     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 5462     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9415     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 5491     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9515     |
----------------------------------
Eval num_timesteps=9520, episode_reward=9.80 +/- 0.40
Episode length: 9.80 +/- 0.40
----------------------------------
| eval/               |          |
|    mean_ep_length   | 9.8      |
|    mean_reward      | 9.8      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 9520     |
------

...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 456      |
|    fps              | 5037     |
|    time_elapsed     | 1        |
|    total_timesteps  | 10015    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 460      |
|    fps              | 5024     |
|    time_elapsed     | 2        |
|    total_timesteps  | 10106    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 464      |
|    fps              | 5038     |
|    time_elapsed     | 2        |
|    total_timesteps  | 10180    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 564      |
|    fps              | 5602     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12539    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 568      |
|    fps              | 5619     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12616    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 572      |
|    fps              | 5641     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12714    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 672      |
|    fps              | 6099     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15051    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 676      |
|    fps              | 6113     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15126    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 680      |
|    fps              | 6131     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15223    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 780      |
|    fps              | 6474     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17581    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 784      |
|    fps              | 6484     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17666    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 788      |
|    fps              | 6497     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17754    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 880      |
|    fps              | 6738     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19751    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 884      |
|    fps              | 6747     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19835    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 888      |
|    fps              | 6763     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19959    |
----------------------------------


In [40]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [41]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()